# Download Data

## Download OSM data

In [1]:
import pyrosm
from pyrosm import get_data

fp = get_data("Toronto", directory = "data/osm")

## Download TTC Transit Feed data

In [2]:
import requests

# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "ttc-routes-and-schedules"}
package = requests.get(url, params = params).json()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

# To get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
        resource_metadata = requests.get(url).json()
        url = resource_metadata["result"]["url"]
        print(resource_metadata)
        # From here, you can use the "url" attribute to download this file


{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=resource_show', 'success': True, 'result': {'cache_last_updated': None, 'cache_url': None, 'created': '2020-12-07T21:50:41.530037', 'datastore_active': False, 'extract_job': 'Airflow: upload_remote_files', 'format': 'ZIP', 'hash': '', 'id': 'cfb6b2b8-6191-41e3-bda1-b175c51148cb', 'is_datastore_cache_file': False, 'is_preview': False, 'last_modified': '2023-09-28T12:00:59', 'metadata_modified': '2023-09-28T13:15:50.193866', 'mimetype': 'application/zip', 'mimetype_inner': None, 'name': 'TTC Routes and Schedules Data', 'package_id': '7795b45e-e65a-4465-81fc-c36b9dfff169', 'position': 0, 'resource_type': None, 'revision_id': '4e00475d-2c1b-421a-ae1c-7686ae77d08f', 'size': 34200336, 'state': 'active', 'url': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/7795b45e-e65a-4465-81fc-c36b9dfff169/resource/cfb6b2b8-6191-41e3-bda1-b175c51148cb/download/opendata_ttc_schedules.zip', 'url_type': 'upload'}}

In [3]:
import requests, zipfile, io

#Download data with API
r = requests.get(url)
with open (r"data/TTC/gtfs.zip", mode="wb") as file:
    file.write(r.content)

#Extract gtfs zipfile
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("analysis/data/ttc")

## Download TTC Subway Lines Shapefiles

In [4]:
import requests

# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "ttc-subway-shapefiles"}
package = requests.get(url, params = params).json()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

    # To get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"] #api URL
        resource_metadata = requests.get(url).json()
        url = resource_metadata["result"]["url"] #result url
        print(resource_metadata)
    # From here, you can use the "url" attribute to download this file

{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=resource_show', 'success': True, 'result': {'cache_last_updated': None, 'cache_url': None, 'created': '2019-07-23T18:13:00.742161', 'datastore_active': False, 'format': 'SHP', 'hash': '', 'id': '7d68bb52-3285-45d7-a248-7748cb47f6ce', 'is_datastore_cache_file': False, 'is_preview': False, 'last_modified': '2019-07-23T18:13:00.719515', 'metadata_modified': '2022-03-25T15:06:23.894145', 'mimetype': 'application/zip', 'mimetype_inner': None, 'name': 'ttc-subway-shapefile-wgs84', 'package_id': 'c01c6d71-de1f-493d-91ba-364ce64884ac', 'position': 0, 'resource_type': None, 'revision_id': '279d3841-1011-43db-b225-692623995917', 'size': 20796, 'state': 'active', 'url': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/c01c6d71-de1f-493d-91ba-364ce64884ac/resource/7d68bb52-3285-45d7-a248-7748cb47f6ce/download/ttc-subway-shapefile-wgs84.zip', 'url_type': 'upload'}}


In [5]:
import requests, zipfile, io

r2 = requests.get(url)
z2 = zipfile.ZipFile(io.BytesIO(r2.content))
z2.extractall("analysis/data/ttc")

# Create Network Dataset (travel-times.py script)

In [1]:
%set_env JAVA_HOME=C:\Program Files\Microsoft\jdk-17.0.8.101-hotspot\

env: JAVA_HOME=C:\Program Files\Microsoft\jdk-17.0.8.101-hotspot\


In [118]:
import geopandas as gpd
import pandas as pd
import numpy as np
import fiona
import r5py
import datetime

C:\Users\teres\AppData\Local\Temp\ipykernel_31164\3027855954.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
Cannot

In [3]:
#Bring in the geojson files for rinks and hex_grid
rinks = gpd.read_file("data/rinks.geojson")

#Data must contain an ID column
rinks["id"] = rinks._id
hex_grid = gpd.read_file("data/hex-grid-centroids.geojson")

In [4]:
#Create transport network
transport_network = r5py.TransportNetwork("data/osm/Toronto.osm.pbf",["data/ttc/gtfs.zip"])

## Get Minimum Travel Time - Walking + Transit

In [5]:
#Compute travel time matrix for walking and transit on a weekday at 6:30pm
from r5py import TransportMode

travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network,origins=rinks,destinations=hex_grid,
    departure=datetime.datetime(2023,10,10,18,30), 
    transport_modes=[TransportMode.WALK, TransportMode.TRANSIT], speed_walking=4.5)

In [6]:
travel_time_matrix = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix.to_csv("data/travel_time/tt_matrix.csv", index = True)

In [7]:
#Get only minimum travel time to the cloest ring for each hex
import pandas as pd

tt = pd.read_csv("data/travel_time/tt_matrix.csv")
tt["travel_time"] = tt["travel_time"].fillna(120)

#Find minimum walking + transit time from hex to the cloest ring
min_tt_wt_630 = tt.groupby("to_id").agg({"travel_time":[min]})
min_tt_wt_630 = min_tt_wt_630.rename(columns ={"travel_time":"walk_&_transit_weekday"})
min_tt_wt_630.to_csv("data/travel_time/access-min-travel-time.csv")

In [8]:
min_tt_wt_630.head()

,walk_&_transit_weekday
,min
to_id,
1544,43.0
1545,43.0
1708,40.0
1709,41.0
1710,41.0


## Get Minimum Travel Time - Walking

In [9]:
#Compute travel time matrix for walking ONLY 
from r5py import TransportMode

travel_time_matrix_computer_walk = r5py.TravelTimeMatrixComputer(
    transport_network,origins=rinks,destinations=hex_grid,
    departure=datetime.datetime(2023,10,10,18,30),
    transport_modes=[TransportMode.WALK], speed_walking=4.5)

In [10]:
travel_time_matrix_walk = travel_time_matrix_computer_walk.compute_travel_times()
travel_time_matrix_walk.to_csv("data/travel_time/tt_matrix_walk.csv", index = True)

In [11]:
import pandas as pd

tt_walk = pd.read_csv("data/travel_time/tt_matrix_walk.csv")
tt_walk["travel_time"] = tt_walk["travel_time"].fillna(120)

#Find minimum walking time from hex to the cloest ring
min_tt_walk = tt_walk.groupby("to_id").agg({"travel_time":[min]})
min_tt_walk = min_tt_walk.rename(columns ={"travel_time":"walk"})
min_tt_walk.to_csv("data/travel_time/min_walking_travel_time")

In [12]:
min_tt_walk.head()

,walk
,min
to_id,
1544,51.0
1545,51.0
1708,47.0
1709,49.0
1710,51.0


## Get Minimum Travel Time - Transit (Weekday)

In [13]:
#Compute travel time matrix for transit ONLY on a weekday at 6:30pm

travel_time_matrix_computer_transit_weekday = r5py.TravelTimeMatrixComputer(
    transport_network,origins=rinks,destinations=hex_grid,
    departure=datetime.datetime(2023,10,10,18,30),
    transport_modes=[TransportMode.TRANSIT], speed_walking=4.5)

In [14]:
travel_time_matrix_computer_transit_weekday = travel_time_matrix_computer_transit_weekday.compute_travel_times()
travel_time_matrix_computer_transit_weekday.to_csv("data/travel_time/tt_matrix_transit_weekday", index = True)

In [15]:
tt_transit_weekday = pd.read_csv("data/travel_time/tt_matrix_transit_weekday")
tt_transit_weekday["travel_time"] = tt_transit_weekday["travel_time"].fillna(120)

#Find minimum transit time from hex to the cloest ring on a weekday
min_tt_transit_weekday = tt_transit_weekday.groupby("to_id").agg({"travel_time":[min]})
min_tt_transit_weekday = min_tt_transit_weekday.rename(columns = {"travel_time":"transit_weekday"})
min_tt_transit_weekday.to_csv("data/travel_time/min_transit_travel_time")

In [16]:
min_tt_transit_weekday.head()

,transit_weekday
,min
to_id,
1544,43.0
1545,43.0
1708,40.0
1709,41.0
1710,41.0


## Get Minimum Travel Time - Transit (Weekend)

In [17]:
#Compute travel time matrix for transit on a weekend at 2:30pm

travel_time_matrix_computer_transit_weekend = r5py.TravelTimeMatrixComputer(
    transport_network,origins=rinks,destinations=hex_grid,
    departure=datetime.datetime(2023,10,14,14,30), 
    transport_modes=[TransportMode.TRANSIT], speed_walking=4.5)

In [18]:
travel_time_matrix_computer_transit_weekend = travel_time_matrix_computer_transit_weekend.compute_travel_times()
travel_time_matrix_computer_transit_weekend.to_csv("data/travel_time/tt_matrix_transit_weekend", index = True)

In [19]:
tt_matrix_transit_weekend = pd.read_csv("data/travel_time/tt_matrix_transit_weekend")
tt_matrix_transit_weekend["travel_time"] = tt_matrix_transit_weekend["travel_time"].fillna(120)

#Find minimum transit time from hex to the cloest ring on a weekend
min_tt_transit_weekend = tt_matrix_transit_weekend.groupby("to_id").agg({"travel_time":[min]})
min_tt_transit_weekend = min_tt_transit_weekend.rename(columns = {"travel_time":"transit_weekend"})
min_tt_transit_weekend.to_csv("data/travel_time/min_tt_transit_weekend")

In [20]:
min_tt_transit_weekend.head()

,transit_weekend
,min
to_id,
1544,45.0
1545,44.0
1708,41.0
1709,43.0
1710,41.0


## Inner Join to Combine all DFs 

In [21]:
#Use inner join to combine all tables into one df
final = pd.concat([min_tt_wt_630,min_tt_walk,min_tt_transit_weekday,min_tt_transit_weekend],
                 axis=1, join="inner").droplevel(1,axis=1).reset_index(drop=False)
#Export final datatable to a csv
final.to_csv("data/travel_time/travel_time_final.csv")

In [22]:
final

,to_id,walk_&_transit_weekday,walk,transit_weekday,transit_weekend
0,1544,43.0,51.0,43.0,45.0
1,1545,43.0,51.0,43.0,44.0
2,1708,40.0,47.0,40.0,41.0
3,1709,41.0,49.0,41.0,43.0
4,1710,41.0,51.0,41.0,41.0
...,...,...,...,...,...
19583,41369,63.0,120.0,63.0,65.0
19584,41370,67.0,120.0,67.0,69.0
19585,41371,69.0,120.0,69.0,70.0
19586,41532,70.0,120.0,70.0,72.0


## Combine Census Variable DFs

In [77]:
import pandas as pd
import numpy as np

In [78]:
#Data file 1 - Median Income
df1 = pd.read_csv("data/census/data.csv")
df1 = df1.set_index(["GeoUID"]).rename(columns={
    "v_CA21_560: Median total income in 2020 among recipients ($)":"Median Income(2020,$)"})
df1.head()

,Type,Region Name,Area (sq km),Population,Dwellings,Households,"Median Income(2020,$)"
GeoUID,,,,,,,
35180018,DA,35180018,6.1125,1573,540,527,50800.0
35180019,DA,35180019,0.2620,709,284,275,43600.0
35180021,DA,35180021,0.1923,490,231,214,43200.0
35180022,DA,35180022,0.2121,407,206,194,43600.0
35180024,DA,35180024,0.3922,486,173,171,51600.0


In [79]:
#Data file 2 - Immigrant and visble minority population
df2 = pd.read_csv("data/census/data (2).csv")
#Rename columns and remove duplicate columns 
df2 = df2.set_index(["GeoUID"]).rename(columns={
    "v_CA21_4410: Immigrants":"# of Immigrants",
    "v_CA21_4875: Total visible minority population":"# of Visible Minority"})
df2 = df2.drop(columns=["Type", "Region Name", "Area (sq km)", "Population ", "Dwellings ", "Households "])
df2.head()

,# of Immigrants,# of Visible Minority
GeoUID,,
35180018,155.0,110.0
35180019,70.0,50.0
35180021,70.0,15.0
35180022,80.0,50.0
35180024,40.0,0.0


In [80]:
#Data file 3-Population in Low Income Household measured by LIM
df3= pd.read_csv("data/census/data(3).csv")
#Rename column, set GEOUID as index, and remove duplicate columns
df3=df3.set_index(["GeoUID"]).rename(columns={
    "v_CA21_1025: In low income based on the Low-income measure, after tax (LIM-AT)":"Low Income Population"
})
df3=df3.drop(columns=["Type", "Region Name", "Area (sq km)", "Population ", "Dwellings ", "Households "])
df3.head()

,Low Income Population
GeoUID,
35180018,60.0
35180019,70.0
35180021,25.0
35180022,35.0
35180024,20.0


In [81]:
census=pd.concat([df1,df2,df3], axis=1)
census.replace(0, np.nan, inplace=True)

#Calculate immigrant and visible miniority percentages
census["Immigrant%"] = census.apply(lambda x: x["# of Immigrants"]/x["Population "]*100, axis=1)
census["Visible Minority%"] = census.apply(lambda x: x["# of Visible Minority"]/x["Population "]*100, axis =1)
census["Low-Income %"]=census.apply(lambda x:x["Low Income Population"]/x["Population "]*100, axis=1)

#Calculate population density, immigrant density, visible minority density
census["Population Density"] = census.apply(lambda x: x["Population "]/x["Area (sq km)"], axis =1)
census["Immigrant Density"] = census.apply(lambda x: x["# of Immigrants"]/x["Area (sq km)"], axis =1)
census["Visible Minority Density"]= census.apply(lambda x: x["# of Visible Minority"]/x["Area (sq km)"], axis =1)

#Clean Column name
census.columns = census.columns.str.replace(" ", "")
census

#Export as a csv file
census.to_csv("data/census/census.csv")
census

,Type,RegionName,Area(sqkm),Population,Dwellings,Households,"MedianIncome(2020,$)",#ofImmigrants,#ofVisibleMinority,LowIncomePopulation,Immigrant%,VisibleMinority%,Low-Income%,PopulationDensity,ImmigrantDensity,VisibleMinorityDensity
GeoUID,,,,,,,,,,,,,,,,
35180018,DA,35180018,6.1125,1573.0,540.0,527.0,50800.0,155.0,110.0,60.0,9.853783,6.993007,3.814367,257.341513,25.357873,17.995910
35180019,DA,35180019,0.2620,709.0,284.0,275.0,43600.0,70.0,50.0,70.0,9.873061,7.052186,9.873061,2706.106870,267.175573,190.839695
35180021,DA,35180021,0.1923,490.0,231.0,214.0,43200.0,70.0,15.0,25.0,14.285714,3.061224,5.102041,2548.101924,364.014561,78.003120
35180022,DA,35180022,0.2121,407.0,206.0,194.0,43600.0,80.0,50.0,35.0,19.656020,12.285012,8.599509,1918.906176,377.180575,235.737860
35180024,DA,35180024,0.3922,486.0,173.0,171.0,51600.0,40.0,NaN,20.0,8.230453,NaN,4.115226,1239.163692,101.988781,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35431423,DA,35431423,0.8142,2285.0,711.0,706.0,48800.0,510.0,625.0,80.0,22.319475,27.352298,3.501094,2806.435765,626.381724,767.624662
35431425,DA,35431425,5.7717,108.0,46.0,41.0,NaN,20.0,NaN,10.0,18.518519,NaN,9.259259,18.711991,3.465184,NaN
35431426,DA,35431426,0.2193,816.0,345.0,336.0,42400.0,120.0,110.0,70.0,14.705882,13.480392,8.578431,3720.930233,547.195622,501.595987


# Joining DA/Census, Hexagon, Rink, Travel Time data

Calculated average population in each hexagon; calculated #population size closest to each rink

## Calculate the average population in each hexagon for all demographic groups

In [82]:
import pandas as pd

In [83]:
#Hexagon-DA join table exported from QGIS
hex_da = pd.read_csv("data/hex_da_join/hex_da_join.csv")
hex_da=hex_da.sort_values(by=["c_RegionName"]).rename(columns={"id":"hex_id"})
hex_da=hex_da.set_index("c_RegionName")

,hex_id
c_RegionName,
35180499,38404
35180928,37072
35191295,34942
35200002,34629
35200002,34465


In [84]:
#Find the number of hexagon in each DA polygon
da_count=hex_da.groupby(by=["c_RegionName"]).size()
da_count = da_count.to_frame(name ="count")

,count
c_RegionName,
35180499,1
35180928,1
35191295,1
35200002,2
35200003,1


In [85]:
#Join the count with the hex-DA join table
hex_da= hex_da.join(da_count, on="c_RegionName",how="left")
hex_da = hex_da.reset_index(drop=False).rename(columns={"c_RegionName":"RegionName"})

#Join the count data with the census table
hex_da = hex_da.join(census.set_index("RegionName"), on="RegionName", validate="m:1")

In [95]:
#Calculate the population size in each of the hexagon
hex_da["Pop_in_Hex"] = hex_da.apply(lambda x:x["Population"]/x["count"], axis=1)
hex_da["Immigrant_in_Hex"] = hex_da.apply(lambda x:x["#ofImmigrants"]/x["count"], axis=1)
hex_da["VM_in_Hex"] = hex_da.apply(lambda x:x["#ofVisibleMinority"]/x["count"], axis=1)
hex_da["LIncome_in_Hex"] = hex_da.apply(lambda x:x["LowIncomePopulation"]/x["count"], axis=1)

#Export to a csv file
hex_da.to_csv("data/hex_da_join/final_hex_da_census.csv")

## Calculate weighted travel time per demographic group

### Create dataframe with travel time and Census Variable

In [147]:
#Spatial join done in QGIS to join the closest rink to each DA polygon
#Clean the rink_da table
rink_da = pd.read_csv("data/rinks_da_join/rink_da_join.csv")
rink_da = rink_da.drop(columns=['a', 'q', 't', 'dw', 'hh', 'id', 'pop', 'name', 'rgid', 'rpid', 'ruid',
       'rguid','n', 'distance', 'feature_x', 'feature_y', 'nearest_x',
       'nearest_y'])

In [182]:
#Bring in the DA-Hex-Census table to review
rink_hex_da = pd.read_csv("data/hex_da_join/final_hex_da_census.csv").drop(columns=["Unnamed: 0"])
rink_hex_da.head()

,RegionName,hex_id,count,Type,Area(sqkm),Population,Dwellings,Households,"MedianIncome(2020,$)",#ofImmigrants,#ofVisibleMinority,LowIncomePopulation,Immigrant%,VisibleMinority%,Low-Income%,PopulationDensity,ImmigrantDensity,VisibleMinorityDensity,Pop_in_Hex,Immigrant_in_Hex,VM_in_Hex,LIncome_in_Hex
0,35180499,38404,1,DA,0.9797,889.0,318.0,304.0,55600.0,275.0,460.0,35.0,30.933633,51.743532,3.937008,907.420639,280.698173,469.531489,889.0,275.0,460.0,35.0
1,35180928,37072,1,DA,23.5528,3799.0,1231.0,1180.0,45600.0,1575.0,2940.0,220.0,41.458278,77.388787,5.790998,161.297171,66.871030,124.825923,3799.0,1575.0,2940.0,220.0
2,35191295,34942,1,DA,14.7069,10353.0,2695.0,2640.0,37600.0,5715.0,9490.0,580.0,55.201391,91.664252,5.602241,703.955286,388.593109,645.275347,10353.0,5715.0,9490.0,580.0
3,35200002,34629,2,DA,0.0504,333.0,106.0,103.0,34000.0,205.0,280.0,35.0,61.561562,84.084084,10.510511,6607.142857,4067.460317,5555.555556,166.5,102.5,140.0,17.5
4,35200002,34465,2,DA,0.0504,333.0,106.0,103.0,34000.0,205.0,280.0,35.0,61.561562,84.084084,10.510511,6607.142857,4067.460317,5555.555556,166.5,102.5,140.0,17.5


In [149]:
#Bring in travel time table
travel_time = pd.read_csv("data/travel_time/travel_time_final.csv")
travel_time = travel_time.drop(columns=["Unnamed: 0"]).rename(columns={"to_id":"hex_id"})
travel_time.head()

#Join rink-hex-da dataset with travel time
rink_hex_da = rink_hex_da.join(travel_time.set_index("hex_id"), on="hex_id", how="left")

#Export data as CSV 
rink_hex_da.to_csv("data/travel_time/rink-hex-da_tt_join_final.csv")

In [150]:
rink_hex_da_tt_join = pd.read_csv("data/travel_time/rink-hex-da_tt_join_final.csv")
rink_hex_da_tt_join = rink_hex_da_tt_join.drop(columns={"Unnamed: 0"})

#Clean the demographic group sizes per hexagon
rink_hex_da_tt_join["Pop_in_Hex"] = rink_hex_da_tt_join["Pop_in_Hex"].fillna(0)
rink_hex_da_tt_join["Immigrant_in_Hex"] = rink_hex_da_tt_join["Immigrant_in_Hex"].fillna(0)
rink_hex_da_tt_join["VM_in_Hex"] = rink_hex_da_tt_join["VM_in_Hex"].fillna(0)
rink_hex_da_tt_join["LIncome_in_Hex"] = rink_hex_da_tt_join["LIncome_in_Hex"].fillna(0)

#Display cleaned table
rink_hex_da_tt_join.head()

,RegionName,hex_id,count,Type,Area(sqkm),Population,Dwellings,Households,"MedianIncome(2020,$)",#ofImmigrants,#ofVisibleMinority,LowIncomePopulation,Immigrant%,VisibleMinority%,Low-Income%,PopulationDensity,ImmigrantDensity,VisibleMinorityDensity,Pop_in_Hex,Immigrant_in_Hex,VM_in_Hex,LIncome_in_Hex,walk_&_transit_weekday,walk,transit_weekday,transit_weekend
0,35180499,38404,1,DA,0.9797,889.0,318.0,304.0,55600.0,275.0,460.0,35.0,30.933633,51.743532,3.937008,907.420639,280.698173,469.531489,889.0,275.0,460.0,35.0,120.0,120.0,120.0,78.0
1,35180928,37072,1,DA,23.5528,3799.0,1231.0,1180.0,45600.0,1575.0,2940.0,220.0,41.458278,77.388787,5.790998,161.297171,66.871030,124.825923,3799.0,1575.0,2940.0,220.0,120.0,120.0,120.0,120.0
2,35191295,34942,1,DA,14.7069,10353.0,2695.0,2640.0,37600.0,5715.0,9490.0,580.0,55.201391,91.664252,5.602241,703.955286,388.593109,645.275347,10353.0,5715.0,9490.0,580.0,120.0,120.0,120.0,120.0
3,35200002,34629,2,DA,0.0504,333.0,106.0,103.0,34000.0,205.0,280.0,35.0,61.561562,84.084084,10.510511,6607.142857,4067.460317,5555.555556,166.5,102.5,140.0,17.5,40.0,113.0,40.0,37.0
4,35200002,34465,2,DA,0.0504,333.0,106.0,103.0,34000.0,205.0,280.0,35.0,61.561562,84.084084,10.510511,6607.142857,4067.460317,5555.555556,166.5,102.5,140.0,17.5,41.0,113.0,41.0,38.0


### Calculate weighted travel time for each population group by mode of transportation

In [176]:
# #Create a function to calculate the population weighted average travel time 
# def popu_wa_tt(df, col):
#     _summation= df["walk_&_transit_weekday"]*df[col]
#     _ssum = _summation.sum(skipna=True, numeric_only=False)
#     _total = df[col].sum(skipna=True, numeric_only=False)
#     _result = _ssum/_total
#     return _result

In [177]:
#Create a function to calculate the population weighted average travel time 
def popu_wa_tt(df, mode, col):
    _summation= df[mode]*df[col]
    _ssum = _summation.sum(skipna=True, numeric_only=False)
    _total = df[col].sum(skipna=True, numeric_only=False)
    _result = _ssum/_total
    return _result

In [180]:
#Create a loop to calculate the weighted travel time for each demographic group

modes = ["walk_&_transit_weekday","walk","transit_weekday","transit_weekend"]
cols = ["Pop_in_Hex", "Immigrant_in_Hex", "VM_in_Hex", "LIncome_in_Hex"]

list_tt = []
for mode in modes:
    for col in cols:
        tt = popu_wa_tt(rink_hex_da_tt_join, mode, col)
        list_tt.append([mode, col, tt])
        
weighted_tt = pd.DataFrame(list_tt, columns=["Mode","Demographics", "Travel_Time"])

In [181]:
#Pivot table to wide form for easier comparison
weighted_tt = weighted_tt.pivot(index="Mode", columns="Demographics", values="Travel_Time")
weighted_tt

Demographics,Immigrant_in_Hex,LIncome_in_Hex,Pop_in_Hex,VM_in_Hex
Mode,,,,
transit_weekday,24.367691,22.532120,23.167715,25.199175
transit_weekend,24.661826,22.797009,23.405522,25.504687
walk,37.496870,33.538626,34.371936,39.776357
walk_&_transit_weekday,24.367691,22.532120,23.167715,25.199175


## Calculate population closest to each rink

In [96]:
#Table created in QGIS by joining the closest rink to the each hexagon by distance
rinks_hex = pd.read_csv("data/rink_hex_join/rink_hex_join.csv")
rinks_hex = rinks_hex.rename(columns={"id":"hex_id",
                                    "_id":"rink_id"})

#Clean the table by dropping QGIS generated columns
rinks_hex = rinks_hex.drop(columns=['n', 'distance', 'feature_x', 'feature_y', 'nearest_x', 'nearest_y'])

In [97]:
#Join rink, hexagon, DA-Census data, and demographic group sizes per hexagon together
rink_hex_da = pd.read_csv("data/hex_da_join/final_hex_da_census.csv")
rink_hex_da = rink_hex_da.join(rinks_hex.set_index("hex_id"), on="hex_id", how="left").drop(columns=["Unnamed: 0"])
rink_hex_da.head()

,RegionName,hex_id,count,Type,Area(sqkm),Population,Dwellings,Households,"MedianIncome(2020,$)",#ofImmigrants,#ofVisibleMinority,LowIncomePopulation,Immigrant%,VisibleMinority%,Low-Income%,PopulationDensity,ImmigrantDensity,VisibleMinorityDensity,Pop_in_Hex,Immigrant_in_Hex,VM_in_Hex,LIncome_in_Hex,rink_id,Parent Asset Name,Asset ID,Asset Name,Community Council Area,Ward
0,35180499,38404,1,DA,0.9797,889.0,318.0,304.0,55600.0,275.0,460.0,35.0,30.933633,51.743532,3.937008,907.420639,280.698173,469.531489,889.0,275.0,460.0,35.0,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
1,35180928,37072,1,DA,23.5528,3799.0,1231.0,1180.0,45600.0,1575.0,2940.0,220.0,41.458278,77.388787,5.790998,161.297171,66.871030,124.825923,3799.0,1575.0,2940.0,220.0,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
2,35191295,34942,1,DA,14.7069,10353.0,2695.0,2640.0,37600.0,5715.0,9490.0,580.0,55.201391,91.664252,5.602241,703.955286,388.593109,645.275347,10353.0,5715.0,9490.0,580.0,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
3,35200002,34629,2,DA,0.0504,333.0,106.0,103.0,34000.0,205.0,280.0,35.0,61.561562,84.084084,10.510511,6607.142857,4067.460317,5555.555556,166.5,102.5,140.0,17.5,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0
4,35200002,34465,2,DA,0.0504,333.0,106.0,103.0,34000.0,205.0,280.0,35.0,61.561562,84.084084,10.510511,6607.142857,4067.460317,5555.555556,166.5,102.5,140.0,17.5,1,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough,21.0


In [126]:
#Sum of population by rink
rink_pop = rink_hex_da.groupby(by=["rink_id"]).agg({"Pop_in_Hex":"sum"}).round({"Pop_in_Hex":0})

#Count the number of rink where it serves more than 100,000 people
rink_overserve = rink_pop["Pop_in_Hex"][rink_pop["Pop_in_Hex"]>100000].to_frame()
rink_overserve = rink_overserve.join(rinks.set_index("_id"), on="rink_id", how="left")
rink_overserve

,Pop_in_Hex,Parent Asset Name,Asset ID,Asset Name,Public Name,Pad Length (ft.),Pad Width (ft.),Ice Pad Type,Boards (Ice Rink),Rink is Lit,Address,Postal Code,Community Council Area,Ward,geometry
rink_id,,,,,,,,,,,,,,,
1,275013.0,Albert Campbell Square,43568,Albert Campbell Square - Outdoor Ice Pad,Scarborough Civic Centre/Albert Campbell Squar...,NaN,NaN,Outdoor Ice Pad,No,Yes,150 Borough Dr,M1P 4N7,Scarborough,21.0,POINT (-79.25763 43.77346)
3,107450.0,Broadlands Community Centre,43575,Broadlands Park - Outdoor Ice Pad,Broadlands Community Centre Outdoor Artificial...,185.0,85.0,Outdoor Ice Pad,Yes,Yes,19 Castlegrove Blvd,M3A 1K9,North York,16.0,POINT (-79.32293 43.74556)
27,100501.0,Joseph Bannon Park,43572,Joseph Bannon Park - Outdoor Ice Pad,Joseph Bannon Park Outdoor Artificial Ice Rink,NaN,NaN,Outdoor Ice Pad,No,Yes,65 Mary Chapman Blvd,M9M 0B7,Etobicoke York,7.0,POINT (-79.53495 43.73207)
31,206472.0,Mccowan District Park,60143,Mccowan District Park - Outdoor Ice Pad,Mccowan District Park Outdoor Artificial Ice R...,NaN,NaN,Outdoor Ice Pad,Yes,Yes,150 Mccowan Rd,M1M 2A4,Scarborough,20.0,POINT (-79.23826 43.73423)
33,109210.0,Mel Lastman Square,43592,Mel Lastman Square - Outdoor Ice Pad,Mel Lastman Square Outdoor Artificial Ice Rink,132.0,58.0,Outdoor Ice Pad,No,Yes,5100 Yonge St,M2N 5V7,North York,18.0,POINT (-79.41409 43.76769)
60,155235.0,Van Horne Park,43570,Van Horne Park - Outdoor Ice Pad,Pleasantview Community Centre Artificial Ice Rink,185.0,85.0,Outdoor Ice Pad,No,Yes,561 Van Horne Ave,M2j 4S8,North York,17.0,POINT (-79.33762 43.78671)


## Travel Time Summary Table

In [1]:
import pandas as pd

In [2]:
rink_hex_da_tt_join = pd.read_csv("data/travel_time/rink-hex-da_tt_join_final.csv")
rink_hex_da_tt_join.head()

,Unnamed: 0,RegionName,hex_id,count,Type,Area(sqkm),Population,Dwellings,Households,"MedianIncome(2020,$)",...,ImmigrantDensity,VisibleMinorityDensity,Pop_in_Hex,Immigrant_in_Hex,VM_in_Hex,LIncome_in_Hex,walk_&_transit_weekday,walk,transit_weekday,transit_weekend
0,0,35180499,38404,1,DA,0.9797,889.0,318.0,304.0,55600.0,...,280.698173,469.531489,889.0,275.0,460.0,35.0,120.0,120.0,120.0,78.0
1,1,35180928,37072,1,DA,23.5528,3799.0,1231.0,1180.0,45600.0,...,66.871030,124.825923,3799.0,1575.0,2940.0,220.0,120.0,120.0,120.0,120.0
2,2,35191295,34942,1,DA,14.7069,10353.0,2695.0,2640.0,37600.0,...,388.593109,645.275347,10353.0,5715.0,9490.0,580.0,120.0,120.0,120.0,120.0
3,3,35200002,34629,2,DA,0.0504,333.0,106.0,103.0,34000.0,...,4067.460317,5555.555556,166.5,102.5,140.0,17.5,40.0,113.0,40.0,37.0
4,4,35200002,34465,2,DA,0.0504,333.0,106.0,103.0,34000.0,...,4067.460317,5555.555556,166.5,102.5,140.0,17.5,41.0,113.0,41.0,38.0


In [3]:
#Create function to categorize interval
def interval (x): 
    if x < 15:
        val = "Under 15 Mins"
    elif 15 <= x <30:
        val = "15-30 Mins"
    elif 30 <= x <45:
        val = "31-45 Mins"
    elif 45 <= x <=60:
        val = "46-60 Mins"
    elif x > 60:
        val = "More than 60 Mins"
    else:
        val = "N/A"
    return val

In [4]:
#Apply function to create new columns
rink_hex_da_tt_join["Travel_Time_Intervel"] = rink_hex_da_tt_join["walk"].apply(interval)
rink_hex_da_tt_join

,Unnamed: 0,RegionName,hex_id,count,Type,Area(sqkm),Population,Dwellings,Households,"MedianIncome(2020,$)",...,VisibleMinorityDensity,Pop_in_Hex,Immigrant_in_Hex,VM_in_Hex,LIncome_in_Hex,walk_&_transit_weekday,walk,transit_weekday,transit_weekend,Travel_Time_Intervel
0,0,35180499,38404,1,DA,0.9797,889.0,318.0,304.0,55600.0,...,469.531489,889.0,275.0,460.0,35.00,120.0,120.0,120.0,78.0,More than 60 Mins
1,1,35180928,37072,1,DA,23.5528,3799.0,1231.0,1180.0,45600.0,...,124.825923,3799.0,1575.0,2940.0,220.00,120.0,120.0,120.0,120.0,More than 60 Mins
2,2,35191295,34942,1,DA,14.7069,10353.0,2695.0,2640.0,37600.0,...,645.275347,10353.0,5715.0,9490.0,580.00,120.0,120.0,120.0,120.0,More than 60 Mins
3,3,35200002,34629,2,DA,0.0504,333.0,106.0,103.0,34000.0,...,5555.555556,166.5,102.5,140.0,17.50,40.0,113.0,40.0,37.0,More than 60 Mins
4,4,35200002,34465,2,DA,0.0504,333.0,106.0,103.0,34000.0,...,5555.555556,166.5,102.5,140.0,17.50,41.0,113.0,41.0,38.0,More than 60 Mins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18293,18293,35211696,4365,4,DA,38.1165,446.0,142.0,134.0,28400.0,...,10.231789,111.5,60.0,97.5,26.25,40.0,61.0,40.0,44.0,More than 60 Mins
18294,18294,35211696,4531,4,DA,38.1165,446.0,142.0,134.0,28400.0,...,10.231789,111.5,60.0,97.5,26.25,24.0,49.0,24.0,29.0,46-60 Mins
18295,18295,35211696,5195,4,DA,38.1165,446.0,142.0,134.0,28400.0,...,10.231789,111.5,60.0,97.5,26.25,26.0,58.0,26.0,26.0,46-60 Mins
18296,18296,35211697,3867,2,DA,5.5728,1332.0,394.0,384.0,36400.0,...,185.723514,666.0,287.5,517.5,87.50,48.0,72.0,48.0,50.0,More than 60 Mins


In [5]:
pd.set_option("display.max_columns", 500)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
rink_hex_da_tt_join

,Unnamed: 0,RegionName,hex_id,count,Type,Area(sqkm),Population,Dwellings,Households,"MedianIncome(2020,$)",#ofImmigrants,#ofVisibleMinority,LowIncomePopulation,Immigrant%,VisibleMinority%,Low-Income%,PopulationDensity,ImmigrantDensity,VisibleMinorityDensity,Pop_in_Hex,Immigrant_in_Hex,VM_in_Hex,LIncome_in_Hex,walk_&_transit_weekday,walk,transit_weekday,transit_weekend,Travel_Time_Intervel
0,0,35180499,38404,1,DA,1.0,889.0,318.0,304.0,55600.0,275.0,460.0,35.0,30.9,51.7,3.9,907.4,280.7,469.5,889.0,275.0,460.0,35.0,120.0,120.0,120.0,78.0,More than 60 Mins
1,1,35180928,37072,1,DA,23.6,3799.0,1231.0,1180.0,45600.0,1575.0,2940.0,220.0,41.5,77.4,5.8,161.3,66.9,124.8,3799.0,1575.0,2940.0,220.0,120.0,120.0,120.0,120.0,More than 60 Mins
2,2,35191295,34942,1,DA,14.7,10353.0,2695.0,2640.0,37600.0,5715.0,9490.0,580.0,55.2,91.7,5.6,704.0,388.6,645.3,10353.0,5715.0,9490.0,580.0,120.0,120.0,120.0,120.0,More than 60 Mins
3,3,35200002,34629,2,DA,0.1,333.0,106.0,103.0,34000.0,205.0,280.0,35.0,61.6,84.1,10.5,6607.1,4067.5,5555.6,166.5,102.5,140.0,17.5,40.0,113.0,40.0,37.0,More than 60 Mins
4,4,35200002,34465,2,DA,0.1,333.0,106.0,103.0,34000.0,205.0,280.0,35.0,61.6,84.1,10.5,6607.1,4067.5,5555.6,166.5,102.5,140.0,17.5,41.0,113.0,41.0,38.0,More than 60 Mins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18293,18293,35211696,4365,4,DA,38.1,446.0,142.0,134.0,28400.0,240.0,390.0,105.0,53.8,87.4,23.5,11.7,6.3,10.2,111.5,60.0,97.5,26.2,40.0,61.0,40.0,44.0,More than 60 Mins
18294,18294,35211696,4531,4,DA,38.1,446.0,142.0,134.0,28400.0,240.0,390.0,105.0,53.8,87.4,23.5,11.7,6.3,10.2,111.5,60.0,97.5,26.2,24.0,49.0,24.0,29.0,46-60 Mins
18295,18295,35211696,5195,4,DA,38.1,446.0,142.0,134.0,28400.0,240.0,390.0,105.0,53.8,87.4,23.5,11.7,6.3,10.2,111.5,60.0,97.5,26.2,26.0,58.0,26.0,26.0,46-60 Mins
18296,18296,35211697,3867,2,DA,5.6,1332.0,394.0,384.0,36400.0,575.0,1035.0,175.0,43.2,77.7,13.1,239.0,103.2,185.7,666.0,287.5,517.5,87.5,48.0,72.0,48.0,50.0,More than 60 Mins


In [6]:
#Groupby Travel Time Intervel
Pop_interval = rink_hex_da_tt_join[["Pop_in_Hex", "Immigrant_in_Hex", "VM_in_Hex", "LIncome_in_Hex", "Travel_Time_Intervel"]]
Pop_interval = Pop_interval.groupby(by=["Travel_Time_Intervel"]).sum()
Pop_interval

,Pop_in_Hex,Immigrant_in_Hex,VM_in_Hex,LIncome_in_Hex
Travel_Time_Intervel,,,,
15-30 Mins,872452.9,360134.0,393497.5,105480.3
31-45 Mins,525788.4,263108.0,311171.0,69827.3
46-60 Mins,369366.1,203348.0,255614.5,49865.0
More than 60 Mins,359979.4,198498.9,279363.0,44022.1
Under 15 Mins,572556.1,217421.0,246793.9,75290.3


In [7]:
#Create function to calcuate the percentage for each population group by travel time category

def precent_category (col):
    col_total = Pop_interval[col].sum()
    list_percent = []
    index = Pop_interval.index.to_list()
    for travel, value in zip(index, Pop_interval[col]):
        percentage = (value/col_total)*100
        list_percent.append([travel, col, percentage])
    return list_percent

In [53]:
#Create final percentage summary table
cols = ["Pop_in_Hex", "Immigrant_in_Hex", "VM_in_Hex", "LIncome_in_Hex"]
final_list = []
for col in cols:
    final = precent_category(col)
    final_list.extend(final)

travel_interval_precent = pd.DataFrame(final_list, columns=["Travel Time", "Demographic Group", "Percentage"])
travel_interval_precent = travel_interval_precent.pivot(index = "Demographic Group", columns="Travel Time", values="Percentage")

#Re-arrange the table for easy reading
travel_interval_precent = travel_interval_precent[['Under 15 Mins', '15-30 Mins', '31-45 Mins',"46-60 Mins", "More than 60 Mins"]]

#Arrange index
new_index=["Pop_in_Hex","Immigrant_in_Hex","LIncome_in_Hex","VM_in_Hex"]
travel_interval_precent = travel_interval_precent.reindex(new_index, axis=0)
travel_interval_precent

Travel Time,Under 15 Mins,15-30 Mins,31-45 Mins,46-60 Mins,More than 60 Mins
Demographic Group,,,,,
Pop_in_Hex,21.2,32.3,19.5,13.7,13.3
Immigrant_in_Hex,17.5,29.0,21.2,16.4,16.0
LIncome_in_Hex,21.9,30.6,20.3,14.5,12.8
VM_in_Hex,16.6,26.5,20.9,17.2,18.8
